In [ ]:
!pip install git+https://github.com/neelnanda-io/Easy-Transformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/neelnanda-io/Easy-Transformer to /tmp/pip-req-build-8i5vxfqy
  Running command git clone -q https://github.com/neelnanda-io/Easy-Transformer /tmp/pip-req-build-8i5vxfqy
     |████████████████████████████████| 41 kB 548 kB/s 
     |████████████████████████████████| 441 kB 51.5 MB/s 
     |████████████████████████████████| 5.5 MB 65.9 MB/s 
     |████████████████████████████████| 1.9 MB 96.2 MB/s 
     |████████████████████████████████| 237 kB 68.4 MB/s 
     |████████████████████████████████| 115 kB 88.7 MB/s 
     |████████████████████████████████| 212 kB 89.0 MB/s 
     |████████████████████████████████| 95 kB 6.1 MB/s 
     |████████████████████████████████| 163 kB 91.0 MB/s 
     |████████████████████████████████| 127 kB 68.7 MB/s 
     |████████████████████████████████| 115 kB 93.2 MB/s 
     |████████████████████████████████| 51 kB 8.1 MB/s 
     |███████

In [ ]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
import tqdm.notebook as tqdm

import random
import time

# from google.colab import drive
from pathlib import Path
import pickle
import os


import matplotlib.pyplot as plt

%matplotlib inline
import plotly.express as px
import plotly.graph_objects as go

from torch.utils.data import DataLoader

from functools import *
import pandas as pd
import gc
import collections
import copy

# import comet_ml
import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets

In [ ]:
from easy_transformer.utils import (
    gelu_new,
    to_numpy,
    get_corner,
    lm_cross_entropy_loss,
)  # Helper functions
from easy_transformer.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from easy_transformer import EasyTransformer, EasyTransformerConfig
import easy_transformer
from easy_transformer.experiments import (
    ExperimentMetric,
    AblationConfig,
    EasyAblation,
    EasyPatching,
    PatchingConfig,
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
from transformers import AutoModelForCausalLM

model_name = "solu-8l-old"
model = EasyTransformer.from_pretrained(model_name).to(device)

Loading model: solu-8l-old


Downloading:   0%|          | 0.00/767 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/828M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/156 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/457k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Using pad_token, but it is not set yet.


Moving model to device:  cuda
Finished loading pretrained model solu-8l-old into EasyTransformer!
Moving model to device:  cuda


## Pruning

In [ ]:
snippets_3_1 = [
    """
"I will. Same to you."

He tells me that he loves me, and then I hang up just as I arrive at Wes's house.

I park in the driveway next to his bike, not bothering to hide my Jeep down the road like usual. I know all about our past, and frankly, now that Wes knows too, his mother's threats are useless. She can't bully me out of Wes's life.

I've made mistakes, not all of them completely my fault, but I can try not to make any more. I'm not sure where we stand relationship-wise, but we'll figure it out. We might even make the right decisions this time. For now, we just have to save the world. No pressure.

I quickly dial Nathan, but he doesn't answer, making me nervous. I tell him to call me back, and that I'm at Wes's house. I push the phone into my pocket and climb out of the Jeep.

Wes waits for me at his door, smiling and looking a little nervous. I wonder if he thought I was going to turn onto the freeway and drive away instead of coming here. The thought didn't even cross my mind. I would have followed him anywhere.

"Parents aren't home," he says, watching my approach. "In case you were worried."

"Now you won't have to lock the door," I say, and stop in front of him.

Wes's smile fades. "I always lock the door," he replies, and turns to push inside his basement apartment.

I realize that I'm nervous too. Beyond the life-altering shit that's about to go down with The Program—I'm here with Wes. And I'm still not entirely sure how to act.

I understand what Michael Realm meant now, how remembering can be a curse. Because I remember things that Wes doesn't. I remember how much he loved me. How much I loved him. The stuff they couldn't take. The stuff that crashed back. So much history, and now it's only mine.

I walk inside his room and close the door behind me. It's dimly lit, the high-set window not enough on a darkened, stormy day. But Wes doesn't flip on the lights as he leads us into the living room area.

I sit on the couch, and Wes comes to the coffee table and turns his laptop in my direction before telling me he'll be right back. He jogs up the stairs and disappears inside his house.

I smile at the wallpaper on his computer, a vintage motorcycle, mid-repair. It's simple, honest. I click open the browser, and his last page pulls up. It's a board called Survivor Rate, and the quick description says it's a forum for survivors of the epidemic. It has over ten thousand members.

I click on the first thread and start to read through, when I hear Wes close the door and lock it before bounding down the stairs. I look up, and he holds out a bag of frozen peas.

"For your head," he says. "I tried to find an aspirin, but my mom won't keep any pills in the house."

"Oh," I say, taking the icy bag from him. "Thank you." It's kind of sweet of him to do that without me asking. I move my legs aside as he scoots past me and drops down onto the couch in his usual spot. I gently press the peas to my head, groaning at the pressure.

"This is the one," Wes starts, turning the screen so he can see it too, "where the guy had the picture of Michael."

"He goes by Realm," I say, and feel Wes turn to me. I point at the screen to move forward. "Can I see the picture?"

"Sure," Wes replies, and clicks into a different thread, scrolling through posts. He double-clicks one. "Here you go. That's him, right?"

And it is. There's a picture of Realm, not looking at the camera. He's partially turned away, his scar prominent on his neck. He doesn't seem to know his picture is being taken, and I'm reminded of Melody and how she never wanted to be in any photos. She always found an excuse. It was probably because she was a handler, a closer, and she didn't want to be recognized. She didn't want a record of being Jana Simms.

"That's definitely him," I say. Under the picture, the post reads: Anyone know this guy?

Wes goes into the private messages and shows me his exchange with the original poster. It doesn't give us any information on locating Realm,
""",
"""
to the human analyte enzyme in its native matrix (eg serum); iii) and the inter-method ratio should be constant (within the limits of experimental error) for the enzyme calibrator and for all patients' samples.<|endoftext|>Q:

Render named children in React component?

<|endoftext|>A man told Dr. Phil on his show yesterday that he might have to kill his own child to protect his wife. His 12-year-old son, Jack, has exhibited traits of “demon possession” since he was two and needed an exorcism, according to the father John.

John, a Christian, went on to say that the demon controlled his son and told him to kill people in the family.

He says he’s so concerned, he has plans for Jack to have an exorcism because he fears that if he doesn’t, one day, he may have to kill his son to protect his wife and children.

I don’t know about you, but this is never a thought that would cross my mind. I would protect my child at all costs, but I guess that’s just what false beliefs do to people. It’s much easier to kill your son if you think he’s an evil devil creature.

Fortunately, Dr. Phil didn’t allow John’s baseless assertion to go unchallenged. He questioned how he could know that Jack is possessed by demons and followed up several times.

“Tell me how you’ve come to the conclusion that it’s demonic possession,” Dr. Phil says to John on Monday’s episode. “At 2 ½-years old, he began exhibiting traits,” John says. “There was the stacking glass around his mother’s neck, point side up, and if she would’ve turned — I mean, I caught him in the middle of it. I felt the energy shift, and I woke up.” “But how is that behavior demonic? “Dr. Phil presses.

It turned out most of the behavior John thought was caused by the devil was pretty typical. The young boy did stack glass around his mother’s neck, and when asked why he did it, he said he was “making mommy pretty.” That’s… normal. Not demonic.

Here are some other reasons John said his son was possessed:

Eyes glazed over, fil
""",
"""
latest lakeshore news & sports for WOMT/WCUB/WQTC/WLKN/WLTU/WEMP

Opening Weekend of Sturgeon Spearing

This past weekend marked the opening weekend for sturgeon spearing on the Winnebago System. Day one was by far the better day, with 219 fish collected. Lake Winnebago saw decent numbers with 83 fish caught, but the Upriver Lakes saw much better success, with 136 fish reeled in. Day 2 was not quite as fruitful, with a total of 124 sturgeon captured. The split was much closer as well, with 56 fish coming from Lake Winnebago, and 68 fish being pulled from the Upriver Lakes. Joseph Orlando has the distinction of catching the largest sturgeon of the weekend, measuring at 77.5” long, and weighing a whopping 147.3 LBS. Congratulations to Joseph, and good luck to those braving the winter cold.<|endoftext|>I am a 44 yr. old french Canadian PHP developper who has been freelancing and offering consulting services for the past 15 years. I have had the opportunity to cater to a diverse range of clients: manufacturers, pharmaceutical companies and media corporations among others.

I am interested in anything related to Open-Source Web developpment.

I am curious on many subjects and aspects of programming: frameworks, design methodologies, testing methodologies, security, analysis and requirement coding tools, and any computer or stat related math.<|endoftext|>Sevoflurane and isoflurane impair edrophonium reversal of vecuronium-induced neuromuscular block.
A dose-response relationship study for edrophonium to examine the modification of volatile anaesthetics on reversal of vecuronium block. One hundred and twenty ASA (I-II) patients were anaesthetized with sevoflurane, isoflurane (1 minimum alveolar anaesthetic concentration [MAC] end-tidal concentration), or fentanyl-diazepam anaesthesia, in combination with 66% nitrous oxide (n = 40 for each group). The evoked electromyogram (EMG) response of the abductor digiti minimi was monitored at 20 sec intervals following train-of-four (TOF) stimulation of the ulnar nerve. The initial neuromuscular block was produced by vecuronium 100 micrograms.kg-1. When the amplitude of the first response (T1) had spontaneously recovered to 10% of the control, edrophonium (0, 125, 400, 700 or 1000 micrograms.kg-1; eight patients each) was randomly administered, and the ratio of the fourth TOF to the first response (TOFR) was monitored at one minute intervals for 10 min. Sevoflurane and isoflurane impaired the edrophonium-assisted TOFR recovery in an edrophonium dose and time dependent manner. The dose-response curves at 10 min exhibited a greater shift to the right in the sevoflurane and isoflurane groups than in the fentanyl-diazepam-nitrous oxide group (P < 0.05). Higher ED50 values (the edrophonium dose required to obtain TOFR value of 50%) in the sevoflurane (> 1000 micrograms.kg-1) and isoflurane groups (851 micrograms.kg-1) were observed than in the fentanyl-diazepam-nitrous oxide group (339 micrograms.kg-1) (P < 0.05). One MAC sevoflurane and isoflurane anaesthesia impair edrophonium reversal of vecuronium block to a similar degree.<|endoftext|>Last weekend, Super Smash Bros. Melee's top talent traveled to Salt Lake City's Vivint Smart Home Arena for three days of high-level competition. Although Game Tyrant Expo (GT-X) had a bracket chock-full of potential winners, nobody could have predicted the extreme upsets that rocked Utah's first major.

It's only fitting that a tournament defined by upsets would finish with one. At the end of the day, Adam "Armada" Lindgren, undoubtedly the favorite to win, was not the player standing atop the podium. Instead, Juan "Hungrybox" Debiedma slugged his way through a grueling 10-game grand finals to claim the largest share of the event's unprecedented $30,000 prize pool.

By capturing back-to-back major titles, Hungrybox has definitively set himself apart from the group of contenders trying to take the number one ranking from Armada. But while the thought of playing further probably didn't cross Hungrybox's mind as he celebrated with a sea of fans, the season is far from over. In less than a week, Melee's pros will pick up their controllers once more to see who can fight their way to the top at The Big House 7. And despite his recent losses, Armada will still b
""",
"""
goodness my boss valued me and my job was safe, but my gut instincts had landed me in more than a little trouble that day and I was left wondering if I was right. The thought did briefly cross my mind that maybe I was over-analysing the situation and perhaps I was just a judgemental meanie!

I had accepted that I'd never know if my intuition was right or if I was way out of line, when several months later Jim wandered into the lounge one afternoon, grinning at me.

'There you go, you can put yourself out of your misery.' As he tossed me the paper he exclaimed, 'You were right!'

As I started to read the article headed 'These little piggies have roast beef ', my jaw dropped. The sweet animal-hugging vegetarian, who had condemned me so harshly for not adopting out our animals to her, had started a free-range home-kill meat business and breeding facility. There she was with her husband, proudly cuddling one of their pigs. The article went into detail about how much they loved their lifestyle, their animals and their new wonderful service to the community.

While I agree that free-range pig farming can be a momentous improvement on the cruel practice of factory farming, I think it's important for people to be honest and ethical in their intentions—a vegetarian pig farmer is somewhat hypocritical in my view!

'Hey Carolyn, I just drove past the circus and there were a whole lot of protesters picketing it.'

Jim was calling me from his cell phone. He made a great roving reporter, and could have had a job as the eye in the sky for the local radio station.

'Is it the elephant do you think?'

We had heard about the controversy surrounding this particular circus and its exploitation of exotic animals. A year had passed since our sneak peek behind the scenes of this very circus, and the animals we had seen had not stopped preying on our minds. As Jim continued to talk, all the unpleasant memories came flooding back.

'What do you think, should we protest with them?' Jim had pulled over to watch the chaos as the big top was being erected in a local park.

'I don't know. I wonder if anyone has actually just talked to him . . . maybe we should try?'

As we sat with the ringmaster, we just listened at first. It was obvious he still loved an audience and he loved to tell his story. His was a classic; he had joined the circus as a boy and 50-odd years later it was all he knew. We asked him what his plans were and he admitted that he was tired and would love to slow down as his health was deteriorating. 'But with the bloody protesters on his back' he was too proud and stubborn to make the move.

Over the next week Jim and I went to the river and foraged for willow branches for the elephant, loaded up Jim's ute and drove our offerings to the circus. We became familiar faces and the team of roadies were always grateful for our help. Before long we were able to talk to the ringmaster about retiring the animals. There was the elephant, the monkeys, the donkeys and pony as well as the lions, poodles and doves. He agreed it was time to let all of them go, except for the poodles and doves which were his wife's much-loved pets. After doing the sums and assessing our land we gave ourselves a reality check; lions and an elephant were probably too much for us to handle and after talking with the ringmaster we knew he had other good offers on the table from established wildlife sanctuaries. No one, however, wanted the monkeys. There were no sensible solutions in the pipeline, so we decided we would put our hands up. We also agreed to take Pablo the pony and Jenny and Wee One the donkeys back to HUHA.

The day we were going to pick up the monkeys came along in a bit of a hurry. The ringmaster wasn't well and needed to move them on faster than agreed. It was winter and the tricky thing was that we didn't have anywhere to put them yet, let alone have an official sign-off from the Ministry of Agriculture and Fisheries (MAF). The ringmaster insisted that they needed to be gone immediately. He also insisted that we pay for the monkeys' caravan and temporary enclosure as part of the deal for which he would accept no less than $20,000. We tried to reason with him but his final words were always, 'Well you can bugger off then, and they can stay here.'

We didn't want to lose the opportunity to get them out of the circus. We knew there was an opportunity for the ringmaster to sell them to another circus, so we weighed up our options and organised a second mortgage on our
""",
"""
&= \iiiint_{\mathbb{R}^{4}}\left(-4\pi \delta(\bar{r} -\bar{r}')\delta(t -t'))\right) f(\bar{r}',t') \,dx'dy'dz'dt'=\\
&= -4\pi \iiiint_{\mathbb{R}^{4}}\delta(\bar{r} -\bar{r}')\delta(t -t') f(\bar{r}',t') \,dx'dy'dz'dt'=\\
&=-4\pi f(\bar{r},t)
\end{aligned}
\end{equation}
\end{mybox}
\begin{tikzpicture}[remember picture, overlay]
\draw[->] (pic cs:start) -- ++(0,-0.5) -- ++(-13,0) |- (pic cs:stop);
\end{tikzpicture}
\end{document}

<|endoftext|>Ultra storage-efficient time digitizer for pseudorandom single photon counter implemented on a field-programmable gate array.
Pseudorandom single photon counting is a novel time-resolved optical measurement method, which is advantageous over convention techniques in terms of data-acquisition speed and system cost. As a critical component of the pseudorandom single photon counter, the photon arriving time digitizer should be storage efficient for a high photon counting rate, while maintaining good time accuracy. We report an ultra storage-efficient time digitizer for a pseudorandom single photon counter in this paper, which is based on the asynchronous serial communication and can store the arriving time of every photon in 1-b memory space. In addition, a novel comb-wave modulator is proposed to achieve the dc balance required for asynchronous serial communication. Our prototype implemented on field-programmable gate arrays provides a time resolution of 400 ps. It can register up to 4.2-Giga photon arriving time tags with 1024 × 32-b memory space.<|endoftext|>I didn't wear my Veronica Mars T-shirt to interview Jason Dohring, but the thought did briefly cross my mind. After all, I was one of the few (well, 2.5 million) viewers who obsessively watched the teen sleuth drama from 2004 to 2007, first on UPN and then The CW.

I integrated Veronica's vernacular into my vocabulary, purchased Mars memorabilia, asked my parents to buy me a T-Mobile Sidekick (Veronica's cellular weapon of choice), and when series creator Rob Thomas asked for my money in 2013 to help fund a feature film, I happily donated.

And I was not alone.

By now, the story of how Veronica Mars became 2013's most talked-about social media event is the stuff of legend. Heck, my father even asked if I donated to "that Pluto movie" (he's trying, guys) when I was home for Thanksgiving. But for those of you who just emerged from underneath a rock, here's the quick version:

On March 12, 2013, Rob Thomas launched a Kickstarter campaign to turn his beloved neo-noir series about a teenage private eye into a movie. He gave himself two days to raise $2 million. That goal was met in 10 hours. But the Kickstarter clock kept counting and the donations continued to pour in, eventually topping out at $5,702,153 (a new Kickstarter record). So, the film was funded, the movie was made, the T-shirts were silkscreened, and that brings us to the present day, when I'm sitting cross-legged on the floor of a Los Angeles office with Jason Dohring — who played Logan Echolls, the show's bad boy with a heart of 24-karat gold — talking about the March 14 release of Veronica Mars' feature film.<|endoftext|><script>
	export let foo = '">div><script>alert(42)' + 'script>';
script>

<div foo={foo}>div><|endoftext|>Q:

Why do Maps have slower lookup property than Records in Erlang?

I'm reading Programming Erlang, in Chapter 5 of the book it says:

Records are just tuples in disguise, so they have the same storage and performance
  characteristics as tuples. Maps use more storage than tuples and have
  slower lookup properties. 

In languages I've learned before, this is not the case. Maps are usually implemented as a Hash Table, so the lookup time complexity is O(1); Records (Tuples with names) are usually implemented as an immutable List, and the lookup time complexity is O(N).
What's different in the implementation of these data structures in Erlang?

A:

There's no real practical performance difference between record lookup and map lookup for small numbers of fields. For
""",
"""
though people recognize that smoking can lead to adverse health consequences, they do not have even a basic understanding of the nature and severity of these consequences.<|endoftext|>Digital Speech Standard

Digital Speech Standard (DSS) is a proprietary compressed digital audio file format defined by the International Voice Association, a co-operative venture by Olympus, Philips and Grundig Business Systems.

DSS was originally developed in 1994 by Grundig with the University of Nuremberg. In 1997, the digital speech standard was released, which was based on the previous codec. It is commonly used on digital dictation recorders. Modern psychoacoustical codecs that perform nearly as well at only slightly higher bitrates have led to this speech coding standard being less used in modern voice recording equipment.

Operation 
The DSS file format stores voice audio data in a highly compressed format that allows basic recording functionality (such as recording, playing, rewinding, etc.) as well as the ability to record in either insert or overwrite mode making it ideal for dictation. This along with ability to include additional information in the file header for the transcriptionist including priority mark, author, job type, etc.

DSS is a format designed specifically for speech, equivalent to MP3 for music. In contrast with MP3, however, the quality usually is as low as possible, to minimize the size of the file.

External links
Philips Dictation Systems- Website of Philips Dictation Systems
Olympus Professional Dictation- Website of Olympus Professional Dictation
Olympus.
Grundig Business Systems - Website of Grundig Business Systems
Express Scribe DSS player and transcription assistant from NCH Software

See also
Speech Processing Solutions
Philips
Olympus
Grundig Business Systems

 
Category:Digital dictation
Category:Speech codecs<|endoftext|>House Speaker Mark Ferrandino, D-Denver, right, says goodbye to partner Greg Wertsch following the passage of Senate Bill 11, which will allow gay couples to form civil unions in Colorado. (Craig F. Walker, The Denver Post)

A bill allowing same-sex couples to form civil unions is on its way to the governor for his signature, but gay-rights activists say they won't stop until they get true equality, which is marriage.

The Colorado House on Tuesday passed the bill 39-26 despite protests from Republicans that the bill faces legal challenges because it doesn't offer religious exemptions.

"We won't get to debate this again here, but we will debate this in a court of law," said Rep. Lori Saine, R-Dacono.

The passage marks the first time in three years the bill has made it through the House, which was controlled by Republicans in the two previous sessions.

Reps. Paul Rosenthal, D-Denver; Joann Ginal, D-Fort Collins; and Dominick Moreno, D-Commerce City hold hands while listening to comments before a vote on civil unions. The three lawmakers are gay. (Craig F. Walker, The Denver Post)

Speaker Mark Ferrandino, a gay Denver Democrat who has carried the bill each year, said Senate Bill 11 is about love, family and equality under the law.

"This wasn't a choice. This is who I am. This is who we are," he said of being gay. "We need to make laws in our society that respect everyone equally."

After the vote, the five gay lawmakers in the House and the three gay lawmakers in the Senate lauded those who decades ago took up the fight for equality or were forced to live in the shadows.

"Today is really a memorial, remembering those who were shamed because they were gay or had AIDS," said Sen. Lucia Guzman, D-Denver.

Sen. Pat Steadman, D-Denver, who has sponsored the civil-unions bill for three years, said its passage is the high point of a decades-long struggle.

"Yet we're not there yet. I don't want anyone to think that we somehow reached the peak," Steadman said. "Civil unions are not marriage. They are something that are separate and distinct and lesser and unequal, and that really is not good enough."

But both he and Ferrandino pointed out that civil unions offer important protections for children and families, and they were thrilled to see the bill finally pass.

Gays cannot marry in Colorado because of a 2006 constitutional amendment that defined marriage as between one man and one woman. Voters passed it 55 percent to 45 percent while defeating a civil-unions-type measure.

Gov. John Hickenlooper, a Democrat and a longtime supporter of gay rights, is expected to sign the bill later this month. The bill becomes law May 1.
"""
]

In [ ]:
snippets_3_2 = [
    r"""
     xmllint only uses the reader interface when passed the --stream option:
$ xmllint --stream --pattern /foo/bar test.xml
Node /foo/bar[1] matches pattern /foo/bar
Node /foo/bar matches pattern /foo/bar

A:

From the xmllint(1) man page:
   --pattern PATTERNVALUE
          Used to exercise the pattern recognition engine, which can be
          used with the reader interface to the parser. It allows to
          select some nodes in the document based on an XPath (subset)
          expression. Used for debugging.

It only understands a subset of XPath and its intention is to aid debugging. The library that does understand XPath fully is libxslt(3) and its command-line tool xsltproc(1).
The ``pattern'' module in libxml "allows to compile and test pattern expressions for nodes either in a tree or based on a parser state" and its documentation lives here: http://xmlsoft.org/html/libxml-pattern.html
Ari.

<|endoftext|>Q:

Choosing the best fit sentence to mail prospective PhD supervisor

I am applying for a PhD scholarship which was posted on the university website and for that, I am writing mail to the professor. I have written about my background, academics, interests and experience. I want to ask him the procedure for applying for the scholarship. I am not a native English speaker so I am not sure that which of the following sentence would be best for that:
Plese let me know about the application procedure
Please let me know how can I apply for the position
Or if there is another way to ask, kindly suggest.

A:

The wording probably doesn't matter too much. You can simply say "I am interested in applying for X scholarship. How does the application procedure work?".
However, before you send the email, make sure you check the details aren't already online somewhere. Also consider if your email would be better addressed to the department administrator rather than the professor, as an administrator is more likely to know all the necessary technical details.

<|endoftext|>Bone marrow microvessel density in chronic myeloproliferative disorders: a study of 115 patients with clinicopathological and molecular correlations.
Philadelphia-negative chronic myeloproliferative disorders (CMD) include polycythemia vera (PV), essential thrombocythemia (ET) and primary myelofibrosis (PMF). Angiogenesis is critical in the pathogenesis of PMF. We studied angiogenesis in 115 patients with CMD (23 PV, 24 ET, 46 PMF, 12 post-PV and 10 post-ET myelofibrosis) by assessment of microvessel density (MVD) in bone marrow (BM). Kruskall-Wallis analysis of variance showed that patients with PMF had significantly higher values of MVD than those with PV (P < 0.001), ET (P < 0.001) and controls (P < 0.001). Mann-Whitney U-test demonstrated that patients with PMF at the prefibrotic stage had significantly higher MVD values than those with ET (P = 0.02). Patients with post-PV myelofibrosis showed significantly higher MVD values than those with PV (P < 0.001), as did patients with post-ET myelofibrosis compared with ET (P < 0.001). In patients with CMD, the multivariate generalized linear regression model showed that the JAK2 (V617F) mutational burden (P = 0.01), serum lactate dehydrogenase level (P = 0.003), and anaemia (P < 0.001) independently correlated with MVD. In summary, this study indicates that assessment of BM angiogenesis, as measured by MVD, may be a useful additional tool in the histopathological definition of CMD.<|endoftext|>Microfabrication conventionally uses photolithography or optical lithography processes for selectively removing parts of a substrate, or parts of a material layer on the substrate. For example, photolithography uses a directed light (radiation) source to transfer a pattern from a photomask (also referred to as a mask or reticle) to a light-sensitive resist material formed on the substrate or material layer, thereby generating an exposure pattern in the resist material. Chemical treatments may then be used to etch or otherwise transfer the exposure pattern in the resist material to the substrate or material layer. More recently, microfabrication has implemented other lithography types, such as charged particle beam lithography, that do not necessitate the intermediary step of creating the mask to transfer or generate an exposure pattern in a resist material. For example, electron beam (e-beam) lithography uses a focused beam of electrons to expose the resist material. Instead of using a mask, e-beam lithography “writes” a pattern directly into an energy-sensitive resist material
    """,
    r"""
     before).
What I don't know is how can I have 32 inputs, since the Raspberry Pi board has less than 32 I/O pins.
I have the impression I need another component for that (like a converter... or something), but again, I don't know what I need to build the electronic part of this project.
How can I connect more inputs to the Raspberry Pi? What would be the best way to build this?

The pedals should work simultaneously as well.

A:

The best way depends on factors only you can determine.
I give a couple of ways.

use two MCP23017, each of which supply 16 digital IO.  They use the I2C bus (pins 3/5, GPIO 2/3) and up to 8 can be used on the bus.
use 4 8-bit input shift registers.  They provide 8 digital inputs each.  They can use the SPI bus and you can daisy chain as many as you want.  If needed you could instead bit bang reading the devices from any GPIO.

The shift registers are probably better for your application.  They are likely to be more responsive as you can drive the SPI bus much faster than the I2C bus.

<|endoftext|>Vadym Karatayev

Vadym Karatayev or Vadim Karataev (born 15 January 1964) is an association footballer from the former Soviet Union.

In 1983 Karatayev took part in the Summer Spartakiad of the Peoples of the USSR in the team of Ukrainian SSR. He also participated in the 1983 FIFA World Youth Championship for the Soviet team.

References

External links
 
 
 

Category:1964 births
Category:Living people
Category:People from Alchevsk
Category:Soviet footballers
Category:Ukrainian footballers
Category:Soviet Top League players
Category:FC Dynamo Kyiv players
Category:FC Chornomorets Odesa players
Category:FC Shakhtar Stakhanov players
Category:FC Krystal Kherson players
Category:FC Zimbru Chișinău players
Category:Wisła Płock players
Category:Hapoel Ashkelon F.C. players
Category:Ukrainian expatriate footballers
Category:Expatriate footballers in Israel
Category:Expatriate footballers in Poland
Category:Association football defenders
Category:Neftçi PFK players<|endoftext|>Q:

Choosing the best fit sentence to mail prospective PhD supervisor

I am applying for a PhD scholarship which was posted on the university website and for that, I am writing mail to the professor. I have written about my background, academics, interests and experience. I want to ask him the procedure for applying for the scholarship. I am not a native English speaker so I am not sure that which of the following sentence would be best for that:
Plese let me know about the application procedure
Please let me know how can I apply for the position
Or if there is another way to ask, kindly suggest.

A:

The wording probably doesn't matter too much. You can simply say "I am interested in applying for X scholarship. How does the application procedure work?".
However, before you send the email, make sure you check the details aren't already online somewhere. Also consider if your email would be better addressed to the department administrator rather than the professor, as an administrator is more likely to know all the necessary technical details.

<|endoftext|>Our first glimpse at components that could be destined for the much-rumored 5.8-inch "iPhone 8" and its 4.7 and 5.5-inch companion devices surfaced this morning in a post on reddit. The images are said to be sourced from a case manufacturer who received them from a glass supplier in China.

The first photo depicts what is said to be the front and back panel of the iPhone 8, with the front panel featuring a super thin bezel around all sides along with a top bar that could perhaps house a front-facing camera, microphone, and speaker. There's been some question on how Apple will handle the front-facing camera and mockup devices and renderings haven't offered a clear picture.



Some design renderings have shown a section at the top of the device similar to the front panel in this image, while others seem to feature a bar that extends fully across the top of the device.

The rear panel features a vertical dual-lens camera and a separate protruding lens component, with no rear Touch ID in sight, in line with rumors suggesting Apple has indeed figured out how to build Touch ID into the display of the device.

A second photo depicts the two alleged iPhone 8 components next to alleged components for the 4.7-inch iPhone 7s and the 5.5-inch iPhone 7s Plus, two devices that are rumored
    """,
    r"""
    correlate these alterations with histological damage. Wistar rats were treated with methotrexate (1.5-3.5 mg/kg) for 3 days to induce mucositis. Intestinal permeability was measured by the urinary excretion rate of lactulose and mannitol following administration by gavage. Intestinal perfusion was performed in vivo for evaluation of water and electrolyte transports. Methotrexate-treated rats lost a significant amount of weight and presented a marked reduction in food intake. Methotrexate induced significant and dose-dependent villous atrophy and elongation of crypts in duodenum, jejunum, and ileum. Methotrexate also induced an increase in sodium and potassium secretion and an important reduction of the mucosa absorptive surface area, shown by the decrease in the mannitol excretion ratio. In conclusion, methotrexate caused major changes in small bowel function by disrupting intestinal permeability and inducing electrolyte secretion in parallel with substantial histological damage.<|endoftext|>Buhay OFW

Buhay OFW (English translation: Life of an OFW) is a weekly public service program aired on 5 Plus catered for the Overseas Filipino Workers or OFWs based in different countries outside the Philippines. The program also featured government and non-government organizations who are charged in taking care the concerns of OFWs such as labor and recruitment issues. Buhay OFW also highlighted untold and successful stories of OFWs who are survived from adversities and sufferings while working outside the country but have also given inspiration to their fellow Kababayans. The program also successfully organized several projects for the OFW community such as the Mr. and Ms. Citizens of the World, which was launched In 2012, in time for the program's 1st anniversary.

The program was premiered on September 10, 2011, and it currently aired as the blocktime program of AksyonTV (5 Plus since 2019) every Saturday evenings at 9-10pm (PST). It is also aired worldwide via AksyonTV International. The program currently hosted by Marissa del Mar, a one-time PMPC Star Awards Best Public Service Program Host awardee and former host of Up Close and Personal on IBC-13, which was awarded as the Best Public Service Program, also from the PMPC Star Awards. Buhay OFW is produced by Millicent Productions, del Mar's own production house.

On October 2016, Buhay OFW was awarded as the Best Public Service Program, together with Mission: Possible of ABS-CBN, in the recently concluded 30th PMPC Star Awards for Television.

References

External links
Official Website

Category:AksyonTV shows
Category:Philippine documentary television series
Category:News5 programs
Category:2011 Philippine television series debuts
Category:2019 Philippine television series endings<|endoftext|>Comparative viability of expanded and unexpanded axial pattern skin flaps in pigs.
A comparison of viable areas of axial pattern flaps post inset was made between expanded and non-expanded pig buttock island flaps. The deep circumflex iliac artery and vein supply approximately the proximal 14 x 10 cm area of this flap. Larger flaps were raised on expanded and control sides of eight pigs to determine if expansion increased the area of survival. In six of eight pigs whose initial tissue expansion did not create more than approximately a 50% increase in skin area, the expanded flaps had a statistically significant increase in viable skin area (proportionally) 10 days post inset than their control flaps (p less than 0.05). Two other pigs did not conform to this pattern. Their initial tissue expansion was greater than 50%, and the resulting area of flap viability was proportionally less in expanded flaps than the control flaps.<|endoftext|>Q:

Question about recommendation letter

I am applying my MA program in October, there is a professor agree to write a recommendation letter for me. But I couldn't find another professor who is willing to write a recommendation letter for me. So I have to apply the mater program next year. I want to email to that professor to tell her I decide not to apply for the MA program this year and ask if she can still help me with the recommendation letter next year (I don't know if she finish it or not). I am not a native English speaker, so I want to ask what should I say to the professor in the email? In a polite way. Thanks!

Can anyone help me to check if this email is polite? Can I say like this?

Hi professor,  I am writing to tell you that unfortunately I met some
  problems with my application for the graduate program, and now I do
  not have enough time for applying my dream school this year.
  Therefore, I decide to take a gap year to participate in some
  internships and prepare for the IELTS test. I will apply the graduate
  program again in September
    """,
    """
     that right? If so, then is it safe to assume that the draft introduction is essentially the "pitch" for the substance of the book? Thanks!

David, I know you've stated the royalty rate hasn't yet been determined, but are there any reasonable assumptions that might be made. For example, will the royalty rate run any higher than it might have in the past to make up for the lack of advance? Is there any sort of ballpark range to help writers determine whether writing a book — or even a proposal — is worth the time spent on the gamble?

Also, what are the terms of copyright/ownership Bloomsbury is purchasing?

What guarantee is there that, once contracted, the book written will ever be published? It seems like without any financial investment in the author, Bloomsbury doesn't have much skin in the game, so to speak... How are you ensuring good-faith with the selected writers?

Mike: Yes, that's a fair assumption, about the draft Intro. Ideally we'd ask for more but it doesn't seem fair to ask people for more when it's entirely possible that we'll receive 200-300 proposals and will only be able to publish 15-20 of them. If we're intrigued by a proposal and want to find out more, we may get back to individual authors at that point.

Anon #1: I can't see that the royalty rate will go any higher than what it's always been on these, which is 10% of net receipts. I'll try to build in some kind of staggered royalty so that the rate goes up after a certain number of sales. Copyright: our contracts all contain, before Clause 1, the phrase "Author is the author of, and owner of the copyright in, the work provisionally entitled..." Good faith: we're nice people. If we sign a contract to work together, we assume that the author will deliver his/her side of the deal, we'll absolutely do the same. Works a treat, every time.

i am not a native speaker, my english is quite good though (i believe). i'm sure i could write a great text for the series. would the possible need for extra proof reading / editing on your end be a problem?

and also: what is the situation with foreign rights? since some of my research for the porposed text would be german language interviews and german is my first language, it would be a reasonably modest job for me to produce a german version of the text. would i be able to sell that directly to a german publisher or would bloomsbury's contract extend to the german version as well?

I got some very positive feedback on a proposal I submitted in the last round (thanks Dave!) but was told the "marketing guys" (okay maybe that was not the term used) were not sold on it because they thought it might not sell enough copies. Should I try again, re-tweaking to suggest why it is in fact a viable idea?

David, My agent believes she should submit my proposal (as she does my work with other publishers), but I don't see how to do that given the carefully choreographed submission procedures. Can an agent submit on behalf of a client?

From the same anonymous: hi again! I should clarify that the "reissue" here is not a reissue of a previously issued compilation but, rather, the first time that three hard-to-find albums were issued as a set. Thanks for any light you can shed on this!

1. Is there any way for you to elaborate on how the royalties work? My understanding is that since there's no advance to recoup, the writer would get royalties starting with the very first copy that sells, correct? Also, does "10% net receipts" mean the author will receive 10% of all sales? Just trying to get an idea. Say the author sold 5,000 copies. How much money would that give them in royalties?

2. Will oral histories be considered for publication? My apologies if there's already been a 33 1/3 oral history published that I haven't read.

Lunt435: No, that would not be a problem, if the proposal is good enough. Foreign rights: Bloomsbury would want to keep these if at all possible, although such matters can be sometimes negotiated at contract stage.

Peg: it's a question we're asked often and there's no easy answer to it! If the project is one you're passionate about, and you feel confident you can persuade us of its viability then yes, please do submit a revised version of it.

Anon #1: yes, an agent can submit your proposal on your behalf, as long as it conforms to the guidelines we've set out.

Anon #2: A collection of short albums
    """
]

In [ ]:
snippets_3_912 = [
    r"""
                super(columns);
            if (type == null || number == null) {
                throw new IllegalArgumentException(
                    "type and number must be non-null");
            }
            this.type = type;
            this.number = number;
            isComparable = (number instanceof Comparable);
        }

        @Override
        @SuppressWarnings("unchecked")
        protected boolean include(
                Entry extends M, ? extends I> value, int index) {
            Object v = value.getValue(index);

            if (v instanceof Number) {
                boolean compared = true;
                int compareResult;
                Class> vClass = v.getClass();
                if (number.getClass() == vClass && isComparable) {
                    compareResult = ((Comparable)number).compareTo(v);
                }
                else {
                    compareResult = longCompare((Number)v);
                }
                switch(type) {
                case BEFORE:
                    return (compareResult > 0);
                case AFTER:
                    return (compareResult < 0);
                case EQUAL:
                    return (compareResult == 0);
                case NOT_EQUAL:
                    return (compareResult != 0);
                default:
                    break;
                }
            }
            return false;
        }

        private int longCompare(Number o) {
            long diff = number.longValue() - o.longValue();

            if (diff < 0) {
                return -1;
            }
            else if (diff > 0) {
                return 1;
            }
            return 0;
        }
    }


    private static class OrFilter<M,I> extends RowFilter<M,I> {
        List<RowFilter super M,? super I>> filters;

        OrFilter(Iterable extends RowFilter super M, ? super I>> filters) {
            this.filters = new ArrayList<RowFilter super M,? super I>>();
            for (RowFilter super M, ? super I> filter : filters) {
                if (filter == null) {
                    throw new IllegalArgumentException(
                        "Filter must be non-null");
                }
                this.filters.add(filter);
            }
        }

        public boolean include(Entry extends M, ? extends I> value) {
            for (RowFilter super M,? super I> filter : filters) {
                if (filter.include(value)) {
                    return true;
                }
            }
            return false;
        }
    }


    private static class AndFilter<M,I> extends OrFilter<M,I> {
        AndFilter(Iterable extends RowFilter super M,? super I>> filters) {
            super(filters);
        }

        public boolean include(Entry extends M, ? extends I> value) {
            for (RowFilter super M,? super I> filter : filters) {
                if (!filter.include(value)) {
                    return false;
                }
            }
            return true;
        }
    }


    private static class NotFilter<M,I> extends RowFilter<M,I> {
        private RowFilter<M,I> filter;

        NotFilter(RowFilter<M,I> filter) {
            if (filter == null) {
                throw new IllegalArgumentException(
                    "filter must be non-null");
            }
            this.filter = filter;
        }

        public boolean include(Entry extends M, ? extends I> value) {
            return !filter.include(value);
        }
    }
}
<|endoftext|>Mesonerilla prospera

Mesonerilla prospera is a species of invertebrate in the Nerillidae family endemic to Bermuda.
In 2000, M. prospera was put on the IUCN Red List under the critically endangered category. The IUCN states that "there are problems with the Order name and the correct placement of the family."

References 

Category:Polychaetes
Category:Endemic fauna of Bermuda
Category:Animals described in 1982
Category:Taxonomy articles created by Polbot<|endoftext|>We have to prepare ourselves for a second referendum

It is hard to think of any phrase in 21st-century Britain that is more Orwellian than ‘People’s Vote’.

The bruised elites, still reeling from the vote for Brexit, say ‘People’s Vote’ when what they really mean is second
    """,
    r"""
     Between 1994 and now, one other person was infected and survived. Though, confusingly, the US Centers for Disease Control reports that two out of three human infections prior to this year were fatal.

Julia Whitty is Mother Jones’ environmental correspondent, lecturer, and 2008 winner of the Kiriyama Prize and the John Burroughs Medal Award.<|endoftext|>Old Hamilton County Jail

The Old Hamilton County Jail is a historic building at 501 Northeast 1st Avenue, Jasper, Florida, United States. It was added to the National Register of Historic Places in 1983.  The Old Hamilton County Jail now serves as the Hamilton County Historical Museum.

Hamilton County Historical Museum

The Hamilton County Historical Museum consists of a jail.

See also
Jails and prisons listed on the National Register of Historic Places

References

External links
 Hamilton County Historical Museum - official site
 Hamilton County listings, Florida's Office of Cultural and Historical Programs

Hamilton County Jail
Category:Museums in Hamilton County, Florida
Hamilton County Jail
Category:Prison museums in Florida
Category:History museums in Florida
Category:Jails in Florida
Category:National Register of Historic Places in Hamilton County, Florida
Category:1893 establishments in Florida<|endoftext|>Q:

Убрать у всех class="active"

Как на js можно у всех li которые в ul убрать класс active, есть идея реализации в цикле пройтись и проверить наличие класса и убрать, но не думаю что лучшее решение, как правильнее сделать?
<ul class="nav navbar-nav">
  <li class="active" id="1" ><a href="/" >9a>li>
  <li class="active" id="2" ><a href="/" >8a>li>
  <li class="active" id="3" ><a href="/" >7a>li>
  <li id="4" ><a href="/" >6a>li>
  <li class="active" id="5" ><a href="/">5a>li>
  <li id="6" ><a href="/" >4a>li>
  <li class="active" id="7"  ><a href="/" >3a>li>
  <li id="8" ><a href="/" >2 a>li>
  <li class="active" id="9" ><a href="/" >1a>li>
 ul>

P.s. active может отсутствовать у некоторых элементов как выше в 8, 6, 4

A:

Проще всего это сделать вот так:
$('ul li.active').removeClass('active');

<|endoftext|>More Like This

Quick Reference

The subphylum chelicerata, of which horseshoe crabs form a small part, is more commonly represented by its terrestrial members – the arachnids (spiders, scorpions, and their relatives). The horseshoe crabs'primordial ...<|endoftext|>Q:

How apply a bitmask to an integer in MySQL?

Here is the problem:
The database Rx30 stores the status of a prescription in a field called 'Rxstatus.' The number they store there is an integer that is never higher than 255. The bits that are set determine whether or not a script is filled.
A representative told us that if 16, 32, or 128 is set, the script is deemed 'unfilled.'
So now what we need to do is take an integer, say '49', and see which bits are set. This needs to be done through MySQL for speed's sake. I understand that 49 is:
32+16+1 = 49

So 16 is set in this field, therefore the script is unfilled.
How (in MySQL) can I take an integer, say 152 for example, and determine which bits of it are set?
Once we determine what's set, if 16, 32, or 128 is set we can deem the script unfilled and produce the correct results.

A:

If you have a test bit mask, e.g. 32+16+1 = 49, and you want to know if any of these are set in Rxstatus, you use:
SELECT Rxstatus & 49 != 0

If you want to know if all of them are set, you use:
SELECT Rxstatus & 49 = 49

To set all the bits from
    """,
    """
    plan for the duration of the work.

April, 2005: Proceedings are instituted in the High Court to prevent residents obstructing the construction of the gas pipeline at Rossport. The High Court grants Shell the right to access private lands in the village for the installation of the pipeline. Meanwhile, it is revealed that Shell employed a company of which it is a substantial shareholder to conduct an ‘independent’ audit of the pipeline’s safety.

June, 2005: Five residents from Rossport are jailed for contempt of court for refusing to obey the High Court order not to interfere with the construction of the Corrib gas pipeline. The men vow to stay in prison until they get justice.

Families and supporters of the Rossport Five commence round the clock picketing at Rossport, Bellinaboy and Glengad: Shell pipeline sites in Mayo.

July, 2005: National rallies held in support of the Rossport Five in Castlebar, Belmullet, Ballina, Dublin.

Natural Resources Minister Noel Dempsey orders a further safety review of the Corrib Gas pipeline with a view to ending the standoff between Shell and local residents in Co Mayo.

Minister Dempsey said Marine and Natural Resources Dept officials had carried out an inspection at the Shell onshore site and subject to further legal advice, it was his view that a serious breach of the consents given to Shell in relation to the pipeline had occurred. He ordered Shell to dismantle three kilometres of gas pipeline that it had assembled in north Mayo.

August, 2005: Marine and Natural Resources Minister, Noel Dempsey, granted Shell permission to lay the 75 kilometres of pipeline from the Corrib Field to the North Mayo coastline.

Shell E&P Ireland defers laying the offshore pipeline for the Corrib gas project until 2006. Shell said the temporary suspension ''will not materially affect the project's completion schedule'', and will ''allow for a period of discussion and dialogue''.

Shell announces that it is to lay off 128 workers at the Corrib gas pipeline in Co Mayo. National rally in support of the Rossport 5 held in Galway. Minister for Communications, Marine and Natural Resources Noel Dempsey announces a further safety review.

September, 2005: Family and supporters visit Norway and meet Statoil and public representatives. Rossport Five appear before Mr Justice Finnegan in the High Court.

1 October, 2005: Thousands rally in support of Rossport Five in Dublin.

12 October, 2005: A two-day public consultation organised by the Department of the Marine is held in Geesala, Co. Mayo

25 October, 2005: Rossport Five appear before Mr Justice Finnegan in the High Court

31 October, 2005: The Minister announced that he had appointed Mr Peter Cassells, a former general secretary of the Irish Congress of Trade Unions, to mediate between Shell E&P and the Rossport residents

11th September - Solitaire forced to withdraw from Broadhaven bay after its pipe laying appartus became damaged. The previousday several people had been arrested while blocking roads and Pat O'Donell and his son had been arrested again to prevent them fishing the bay.

Search

Blast from the Past

Garda whistleblower Maurice McCabe’s first contact with a TD came about because he saw Clare Daly TD on ‘Tonight with Vincent Browne’ talking about policing of Corrib Gas protests, writes William Hederman

The repercussions for Garda whistleblowers Maurice McCabe and John Wilson will be familiar to others who have publicly embarrassed An Garda Síochána. They were clearly acting in the public interest, but their revelations brought the force into disrepute, and the two men suffered as a result. Revenge was exacted – not only by colleagues, but also by way of public denunciation by the Garda Commissioner (“disgusting”), the Minister for Justice (“not co-operating”) and by various other parties loyal to the force.<|endoftext|>Allendale County Courthouse

Allendale County Courthouse is a historic county courthouse in Allendale, Allendale County, South Carolina. It was added to the National Register of Historic Places in 2007.

Description and history
It was built in 1921-1922, and is a two-story yellow brick and limestone-accented building with a central block with pedestaled pediment dominated by a monumental, unengaged, flat-roofed Neoclassical Revival portico. The portico features four massive limestone columns and responding pilasters of the Tuscan order, a classical entablature, and a brick-and-limestone parapet.

Immediately to the rear and connected to the historic courthouse by a narrow two-story hyphen is a large office and courtroom building that was completed and occupied in 2004
    """,
    r"""
    <|endoftext|>TimerEventDefinition.class);
    timerDefinition.setTimeDate(timeDate);
    return timerDefinition;
  }

  protected TimerEventDefinition createTimeDuration(String timerDuration) {
    TimeDuration timeDuration = createInstance(TimeDuration.class);
    timeDuration.setTextContent(timerDuration);
    TimerEventDefinition timerDefinition = createInstance(TimerEventDefinition.class);
    timerDefinition.setTimeDuration(timeDuration);
    return timerDefinition;
  }
}
<|endoftext|>China Grove (Gardner, Louisiana)

China Grove is a historic house located in Gardner, Louisiana. It was added to the National Register of Historic Places on December 5, 1984. The house is considered to be an outstanding example of neo-classical and Greek Revival architecture; its Greek Revival woodwork in particular stands out among houses in the region.

It was listed as one result of a study of 10 Neo-Classical farm-plantation houses along Bayou Rapides.  As for several of the others (Eden, Geneva, Hope, Island Home, Longview), China Grove was modified by addition of hood along its original gallery, termed a false gallery, which provides additional protection from the rain, detracting somewhat but not greatly from its original appearance.

References

Category:Houses on the National Register of Historic Places in Louisiana
Category:Houses completed in 1857
Category:Houses in Rapides Parish, Louisiana
Category:Neoclassical architecture in Louisiana
Category:National Register of Historic Places in Rapides Parish, Louisiana
Category:1857 establishments in Louisiana<|endoftext|>Travis Dyson is the 30-year-old Florida man who was found in a hotel room with Democratic politician Andrew Gillum on March 13. Police said Dyson appeared to have suffered a drug overdose. He is conscious and in stable condition in a Miami-area hospital.

Dyson maintains a profile on a website for male escorts and is studying at a nurse practitioner school. Gillum has been married to R. Jai Gillum since 2009. The couple have three children together.

In 2018, Gillum, 40, was the surprise winner of the Democratic primary for governor in Florida. Gillum lost the general election narrowly to Ron De Santis. Since then, Gillum has been a rising star in the Democratic party. Between 2014 and 2018, Gillum served as the mayor of Tallahassee.

Here’s what you need to know:

1. Gillum Said He Was in Miami to Celebrate a Wedding but Dyson Said the Former Mayor Never Mentioned it

Former Gubernatorial Candidate Andrew Gillum Found In Room With Possible Crystal MethAndrew Gillum said he was in town for a wedding and had too much to drink but never used methamphetamines 2020-03-13T16:44:08.000Z

Gillum has said that he was in Miami for a wedding. The former Tallahassee mayor admitted to “drinking too much” but denied that he had taken any drugs. Gillum was too drunk to speak to the police when they arrived at the scene, documents say. Speaking to the Miami New Times, Dyson said that Gillum did not mention attending a wedding. In his statement, Gillum referred to Dyson as a “friend.” Dyson said that he and Gillum had been friends since last spring. Dyson told the website, “I personally was not celebrating a wedding. I don’t know if [Gillum] was in town for a wedding. He did not mention that.”

Former Florida governor candidate Andrew Gillum involved in Miami meth overdose 2020-03-13T16:41:12.000Z

The man who rented the room, Aldo Mejias, 53, said he went to the room before midnight on March 12. Mejias said that he found Dyson and Gillum “under the influence of an unknown substance.” Mejias said that Gillum had been vomiting into the toilet and that Dyson was unconscious. Mejias performed CPR on Dyson until the paramedics arrived. The incident occurred at the Mondrian South Beach Hotel in Miami Beach, Florida.

The drug in question was crystal meth. Those narcotics were impounded. It’s not clear if Gillum or Dyson will be facing any charges. Gillum was allowed to leave the scene and return home by officers.

2. Dyson Was Last in Trouble With the Law in August 2019

Dyson was arrested in August 2019 and accused of resisting officer without violence to his person reckless driving.

On his Instagram page, which was set to private in the fallout from the Gillum scandal, Dyson referred to himself as “still young,” “sometimes professional,” “very taken” and a “grad student.” Dyson says that he studying at nurse
    """,
    r"""
    , audio equipment, tripods, batteries, etc. Works in all kinds of weather conditions.
Statement about Other Duties:
The foregoing is not necessarily an exhaustive list of all functions essential to the job for which the employee is responsible, nor an exhaustive list of the minimum requirements and specifications necessary to perform the essential functions, including all responsibilities, skills, duties, requirements, efforts, or working conditions associated with the job. While this is intended to be an accurate reflection of the current job, management reserves the right to revise the job or to require that other or different functions be performed when circumstances change or exigencies require it.
Interested parties should submit resume, reel (links) and references to Robby Ferguson, Chief Photographer, KFSM-TV via e-mail (preferred method) to robby.ferguson@kfsm.com or by mail to 4201 North Shiloh Dr. Suite 169, Fayetteville, AR 72703<|endoftext|>J.H. Haag House

J.H. Haag House is a historic home located at Garrett, DeKalb County, Indiana.  It was built about 1875, and is a two-story, Italianate-style brick dwelling. It has a cross gable roof and two-story gabled wing.

It was added to the National Register of Historic Places in 1983.

References

Category:Houses on the National Register of Historic Places in Indiana
Category:Italianate architecture in Indiana
Category:Houses completed in 1875
Category:Houses in DeKalb County, Indiana
Category:National Register of Historic Places in DeKalb County, Indiana<|endoftext|>B.C. Premier Christy Clark says 2012 has been a challenging year but she's confident of the road ahead, including her party's chances in the coming provincial election.

Clark said despite trailing the opposition by more than a dozen points in the polls for most of the year and a number of high-profile resignations in her cabinet, she feels good about what she has accomplished in 2012.

"It's been busy. It's been challenging. Probably in many ways it's been the best year of my life," Clark told CBC News Vancouver host Gloria Macarenko during a year-end interview.

The premier said a big part of that confidence has come from focusing on her own agenda with her jobs plan, as opposed to continuing what was started by her predecessor.

"We've really been living that plan and so that has been something of my making, of my government's making."

As for 2013 and the upcoming May provincial election, Clark isn't making any bold predictions but says she is optimistic about her party's chances.

"I'm confident when people sit down and really ask themselves the question about where they want their economy to go, that they'll look at the plan that we've delivered and say, 'You know what? It wasn't perfect, but they got the big things right.'"

Clark was elected as the leader of the B.C. Liberal Party in February 2011. The upcoming May 14 provincial election will be her first as the party's leader.

Also on HuffPost:

Close

���

Who Is Canada's Least Popular Premier?

of

���

���

Angus Reid Public Opinion surveyed 6,657 Canadian adults from August 21 to August 27, 2012. The margin of error is +/- 1.2 per cent, 19 times out of 20.<|endoftext|>Spatial cognition in children. II. Visuospatial and constructional skills in developmental reading disability.
Cognitive models for developmental dyslexia are nowadays centered on the hypothesis of a specific deficit within the phonologic module of the language system. To ascertain whether defects of spatial cognition are associated with developmental reading disability, we investigated a sample of 43 school children (aged 8-9 years) found to be reading impaired during a wide screening survey for developmental dyslexia in the province of Naples, Italy. After one year all children were tested again and only 9/43 still presented reading impairment, while the remaining had achieved a variable range of spontaneous recovery. A detailed analysis was performed on all children to characterize their cognitive performances using on one hand classical conventional tests for constructional praxis, visuospatial cognition, and visuospatial memory and on the other a specific neuropsychological battery for constructional disorders. The results of our study demonstrated that children with long-lasting reading impairment exhibited normal performances on spatial cognition tasks. Moreover, one single child was found with relevant visuospatial deficits pointing to the possible existence of a visuospatial subtype for developmental dyslexia.<|endoftext|>Venice, Day 2

We had spend a lot of time people watching and gelato eating in Piazza San Marco, but this morning we decided to head up the Campanile Bell Tower (thankfully there was
    """
]

In [ ]:
snippets_4_912 = [
    r"""
    Conclusion
==========

Phenformin significantly inhibited both the development and growth of established MCF7 and MDAMB231 tumours without murine toxicity. The potential for phenformin should be considered further as an antineoplastic agent of greater *in vivo* efficacy than metformin for the treatment of breast cancer.

We would like to thank the Breast Cancer Research, Scotland, for financial support.

This work is published under the standard license to publish agreement. After 12 months the work will become freely available and the license terms will switch to a Creative Commons Attribution-NonCommercial-Share Alike 3.0 Unported License.

![Effect of 300 mg kg^−1^ phenformin on human breast MCF7 (**A**) and MDAMB231 (**B**), and 300 mg kg^−1^ metformin on MDAMB231 (**C**) tumour xenografts. For the phenformin experiment, mice were divided into three groups. Controls received no phenformin. Pre-treatment mice were given phenformin (300 mg kg^−1^) in 5% sucrose instead of normal drinking water for 2 weeks prior to injection of MCF7 or MDAMB231 cells. The phenformin group received normal drinking water until tumours reached ���30 mm^3^, after which drinking water was replaced with 5% sucrose containing phenformin (300 mg kg^−1^). Control group also had water replaced with 5% sucrose. For the metformin experiment, mice were divided into two groups. Controls received no metformin and mice with established tumours received metformin (300 mg kg^−1^) in water. MCF7 tumours pre-treated or treated with phenformin had statistically significant inhibition of tumour growth of 88% relative to the control group (*P*\<0.05). Animals injected with MDAMB231 cells and treated prophylactically with phenformin showed small lumps 6 weeks after inoculation, which remained static for the rest of the experiment. Established MDAMB231 tumours treated with phenformin demonstrated statistically significant inhibition of tumour growth of 60% relative to the control group (*P*\<0.05). There were no statistically significant differences between control mice and mice treated with metformin.](bjc201256f1){#fig1}

![Connective tissue histological analysis of MCF7 and MDAMB231 xenografts treated with phenformin using Van Gieson stain. MCF7 tumours treated with phenformin showed substantial increase of connective tissue, demonstrated as red/pink staining, which replaces epithelial tumour cells (**A** and **B**). No differences were observed in MDAMB231 tumours (**C** and **D**), × 5 magnified image captured with an Aperio ScanScope XT, Aperio Technologies, Vista, CA, USA.](bjc201256f2){#fig2}

![Phospho-histone H3 immunohistochemistry analysis of MCF7 and MDAMB231 xenografts treated with phenformin. Tumours were harvested and immunohistochemistry analysis performed as described in Materials and methods. No significant differences were observed for phenformin (**B** and **D**) compared with untreated MCF7 and MDAMB231 tumours (**A** and **C**), × 5 magnified image captured with an Aperio ScanScope XT, Aperio Technologies.](bjc201256f3){#fig3}

![Cleaved PARP analysis of MCF7 and MDAMB231 xenografts treated with phenformin. Tumours were harvested and immunohistochemistry analysis performed as described in Materials and methods. No significant differences were observed for phenformin (**B** and **D**) compared with untreated MCF7 and MDAMB231 tumours (**A** and **C**), × 5 magnified image captured with an Aperio ScanScope XT, Aperio Technologies.](bjc201256f4){#fig4}

![Ki67 immunohistochemistry analysis of MCF7 and MDAMB231 xenografts treated with phenformin. Tumours were harvested and immunohistochemistry analysis performed as described in Materials and methods. No significant differences were observed for MCF7 tumour treated with phenformin compared with control (**A** and **B**). Similar results were found for MDAMB231 tumours (**C** and **D**), × 5 magnified image captured with an Aperio ScanScope XT, Aperio Technologies.](bjc201256f5){#fig5}

![AMPK activation in liver and spleen of mice. Liver and spleen of control mice and mice that were pre-treated or treated after xenograft establishment with 300 mg kg^−1^ phenformin were processed and western blots produced as described in Materials and methods. Lanes 1--4 show the results for control mice, lanes 5--9 show the results for mice pre-treated with phenformin and lanes 10--13 show the results for mice treated with phenformin. Phosphorylation of the activation loop of AMPK (T172) was enhanced in liver and spleen from mice pre-treated or treated (lanes 10--13) with phenformin
    """,
    r"""
    10 days in increasing doses of MMC. Average and s.d. of at least three repeats is shown. (**B**) Survival as measured by MTT assay of short-term primary UM cells after 14 days in 50 n MMC. Cells were extracted from short-term primary UM cultures and tested before passage 5. Survival fraction is the OD value in MMC/the OD value seen in the same cell line grown for 14 days without treatment. Average and s.d. of three repeats are shown, except for two primary cultures where limited passages meant that one repeat was completed, in this case the value obtained from one repeat is shown without error bars.](bjc201156f1){#fig1}

![Mitomycin C induces fewer DNA ICLs in UM. (**A**) Average TM in UM (SOM 157d, SOM 196b) and control cell lines untreated, treated with 10 Gy IR or pretreated with 150 *μ* MMC for 1 h before being treated with IR. The average TM was calculated using CometScore software where at least 50 cells were analysed on each of three occasions and the s.d. is shown. Significance was determined using the Student\'s *t*-test where *n*=3 and *P*\<0.01 is indicated by ^\*\*^. (**B**) MMC-induced percentage decrease in migration calculated for each cell line. The decrease is directly proportional to the amount of ICLs formed. Average and s.d. are shown for at least two repeats. (**C**) Representative COMET assay images for each of the treatments. Images were obtained using the full spectrum function of the CometScore computer software. Cells were originally stained with SYBR Safe DNA gel stain.](bjc201156f2){#fig2}

![Mitomycin C induces fewer *γ-*H2AX foci and less cell cycle arrest in UM. (**A**) Quantification of *γ-*H2AX foci formation in UM (SOM 157d, SOM 196b) and control cell lines with and without incubation in 90 n MMC for 1 h. Average and s.d. of three repeats is shown. Significance was calculated using the Student\'s *t*-test where *n*=3 and *P*\<0.001 is indicated by ^\*\*\*^. (**B**) Percentage of cells in G2-phase of the cell cycle with or without incubation in 90 n MMC for 24 h as measured by PI staining. Average and s.d. of three repeats is shown. Significance was calculated using the Student\'s *t*-test where *n*=3 and ^\*^*P*\<0.05.](bjc201156f3){#fig3}

![Uveal melanoma exhibit reduced expression of CYP450R. (**A**) Western blot for cytochrome p450 reductase (CYP450R), DTD (NQO1) and *β*-ACTIN protein expression in UM (SOM 157d, SOM 196b) and control cell lines. (**B**) Western blot for CYP450R and *β*-ACTIN protein expression in cells extracted from primary UM short-term cultures and tested before passage 5.](bjc201156f4){#fig4}

![Complementation of UM cell lines with CYP450R increases sensitivity to MMC. Western blot for cytochrome p450 reductase (CYP450R) and *β*-ACTIN protein expression in (**A**) control (WM793) and (**C**) UM (SOM 196b) cell lines 48 h after transfection with or without a plasmid expressing CYP450R. (**B** and **D**) Clonogenic survival of cell lines shown above after 10 days in increasing doses of MMC. Average and s.d. of at least three repeats is shown.](bjc201156f5){#fig5}

![Uveal melanoma is resistant to cisplatin but not to HU. Clonogenic survival of UM (SOM 157d, SOM 196b) and control cell lines after 10 days in increasing doses of (**A**) cisplatin and (**B**) HU. Average and s.d. of at least three repeats is shown.](bjc201156f6){#fig6}

###### Clinicopathological details for patients with primary uveal melanoma, and correlation with genetic markers of poor prognosis

  **SOM**    **Sex**  **Cell type**          **Tumour location**   **Copy number for chromosomes 3 and 8 respectively, as determined by FISH[a](#t1-fn2){ref-type="fn"}**
  --------- --------- ---------------------- --------------------- --------------------------------------------------------------------------------------------------------
  SOM520        F     Cilary body            Spindle               2 : 2 Good prognosis
  SOM524        F     Ciliary body           Mixed                 2 : 3 Poor prognosis
  SOM526        F     Ciliary body           Mixed                 2 :
    """
]

In [ ]:
snippets_4_0 = [
    r"""
     warped product manifolds (Deszcz [@Des91], theorem 3 and corollary 1). It was established by Grycak for non-conformally flat pseudo-Riemannian manifolds of dimension $n\ge 5$ [@Grycak]. In $n=4$ it holds for Lorentzian manifolds (Eriksson and Senovilla [@Eriksson]). In $n\ge 5$, by the above equivalence, conformal semi-symmetry implies Ricci semi-symmetry, $[\nabla_i,\nabla_j]R_{kl}=0$. A summation over cyclic permutations of indices $ijk$ gives the algebraic property (Defever and Deszcz [@Defever]): $$R_{im} R_{jkl}{}^m +R_{jm} R_{kil}{}^m +R_{km} R_{ijl}{}^m =0. \label{eq2.1RR}$$ This equation originally appeared in Roter’s paper on conformally symmetric spaces ([@Rot72] lemma 1). In ref. [@ManMol2012] the property was generalized by us, to define the notion of K-compatible tensor, where $K$ is Riemann’s or Weyl’s or a generalized curvature tensor (then eq. states that the Ricci tensor is Riemann compatible). Geometric implications of compatibility were explored for the Riemann tensor [@RCT] and the Weyl tensor [@WCT]. In particular we showed that Weyl and Riemann compatibility are equivalent for the Ricci tensor. Manifolds whose Ricci tensor is Weyl compatible are termed “Weyl compatible manifolds” [@ManSuh2014]. Several such manifolds, which include the Robertson-Walker space-time, were discussed by Deszcz et al. [@Deszcz2013]; another example is Gödel’s metric ([@Deszcz2014], th.2).\
In this paper we present new results on (CR)$_n$ manifolds with dimension $n\ge 5$. In Section 2 we specialize the second Bianchi identity for the Weyl tensor and an identity by Lovelock to algebraic ones, and show that in $n\ge 5 $ the tensor $\alpha_i\alpha_j$ is Weyl compatible; also the Ricci tensor is such, when $\alpha $ is closed. The Weyl tensor is represented as a Kulkarni-Nomizu product in terms of the recurrence vector $\alpha_i$ and the symmetric tensor $E_{il}= C_{ijkl}\alpha^j\alpha^k/\alpha^2 $.\
In Section 3 we introduce the tensor $h_{ij}=C_i{}^{klm}C_{jklm}/C^2$. Among other properties, we show that it is covariantly constant, $\nabla_k h_{ij}=0$. Therefore, this tensor gives an explicit realization of Gricak’s theorem [@Grycak1981], namely that the traceless part of the Ricci tensor is proportional to the traceless part of $h_{ij}$ via a scalar function, eq.. This means that, up to a scalar function, the traceless part of the Ricci tensor is determined by the Weyl tensor. Proportionality implies unicity of $h_{ij}$ and that the Ricci tensor cannot have more than 2 eigenvalues, and that the recurrence vector is an eigenvector.\
In Sections 4 and 5 we focus on Lorentzian (CR)$_n$ manifolds, $n\ge 5$: the existence of $h_{ij}$ with null covariant derivative implies that a (CR)$_n$ Lorentzian manifold is either decomposable or it admits a unique null covariantly constant vector field [@Senovilla; @Hall91]. If $C^2\neq 0$ the manifold is decomposable. A Brinkmann or pp-wave metric requires $C^2=0$, with the characterization given by Galaev [@Galaev].\
With null recurrence vector $\alpha^2=0$, the algebraic type of the Weyl tensor is at least II$_d$ in the high-dimensional Bel-Debever-Ortaggio classification [@Ortaggio09]. This extends the result of Mc Lenaghan and Leroy [@LenaghLer] valid in $n=4$. If $\alpha^2 < 0$, Weyl’s tensor is purely electric, according to the definition given in [@HeOrWy2013].\
In Section 6 we construct a simple example of (CR)$_n$ manifold.

We assume that the manifolds are smooth, connected, Hausdorff, with non-degenerate metric ($n$-dimensional pseudo-Riemannian manifolds) and that $\nabla_j g_{kl} = 0$. Where necessary we specialize to a metric with signature $n-2$ (Lorentzian manifolds).

**A representation of the Weyl tensor**
=======================================

Our presentation largely relies on two general identities for the Weyl tensor on a pseudo-Riemannian

    """,

    r"""
     as $$\begin{aligned}
    ds^2=dr^2+\sin^2(r)g_r,\end{aligned}$$ where $g_r$ the standard round metric on $\mathbb S^n$. We have $$\begin{aligned}
    r_{\tilde{\alpha}\tilde{\beta}}=Hess (r)(e_{\tilde{\alpha}},e_{\tilde{\beta}})=\frac{2\cos(r)}{\sin(r)}g_r(e_{\tilde{\alpha}},e_{\tilde{\beta}}).\end{aligned}$$ thus $$\begin{aligned}
    \Phi_{\alpha\beta}\geq 2\rho_\alpha\rho_\beta+\frac{4\rho\cos(\rho)}{\sin(\rho)}g_r(e_{\tilde{\alpha}},e_{\tilde{\beta}}).\end{aligned}$$ Since $\rho\le \tilde R<\frac{\pi}{2}$, $$\begin{aligned}
    \Phi_{\alpha\beta}(x) \geq C_0 \bar g_{\alpha\beta}(x), \quad x\in B_{\tilde R}(p),\end{aligned}$$ for some constant $C_0>0$.

    In both cases, restricting $\bar \nabla^2 \Phi$ to $M$, we have $$\begin{aligned}
    \Phi_{ij}\geq C_0g_{ij}-C_2h_{ij}.\end{aligned}$$ with some constant $C_2>0$.

If $N$ is an even-dimensional, compact, simply connected, with positive sectional curvature $0<K_N\le k$, the Klingenberg’s injectivity radius estimate [@K] states that $inj(p)\ge \frac{\pi}{\sqrt{k}}, \forall p\in N$. In this case, $\min\{inj(p), \frac{\pi}{2\sqrt{k}}\}=\frac{\pi}{2\sqrt{k}}$.

If $N$ is a Hadamard space, condition (2) in Theorem \[thm2\] is automatically satisfied for any $p\in N$ and $r>0$. When $n=2$ and under the assumption that there is $\delta>0$ such that $$\begin{aligned}
R(x) \ge \delta+\bar{R}(X(x))-2\inf\{\bar{Ric}_{X(x)}(\mu,\mu):\mu \in T_{X(x)}U,|\mu|=1\}, \forall x\in \mathbb S^2,\end{aligned}$$ estimate (\[estH\]) was proved by Pogorelov (Theorem in page 401, [@Pb]) for embedded strictly convex surfaces $(\mathbb S^2, g)$ in $3$-dimensional Hadamard space, where the constant $C$ in (\[estH\]) in addition depends on $\delta$ and inner radius of the domain enclosed by the embedded surface. A lower bound of inner radius of convex surfaces in $\mathbb R^3$ with extrinsic Gauss curvature bounded between two positive constants $K_0, K_1$ follows from a classical result of Blaschke. Such lower bound for inner radius was claimed in [@Pb] for bounded convex surfaces in general $3$-dimensional Hadamard space.

Estimate (\[estH\]) in Theorem \[thm1\] depends on the position of the embedding. It is desirable to replace it by intrinsic diameter of $M$. In the case of space forms, one may achieve this by shifting the origin. From the proof of Theorem \[thm2\], we may replace it by distance function in some cases listed there. We state it as a corollary.

\[cor-3\] If $X: (M^n,g)\to B_{R}(p)\subset  (N, \bar g)$ is a $C^4$ immersed compact oriented hypersurface with nonnegative extrinsic scalar curvature. Suppose for some $k\ge 0$, $K_N(X)\leq k, \forall x\in B_{R}(p)$ and $R\le \min\{inj(p), \frac{\pi}{2\sqrt{k}} \}$, then there’s a constant $C$ depending only on $n$, $\|g\|_{C^4}$, $\|\bar{g}\|_{C^4}$, $\frac{\pi}{2\sqrt{k}}-\sup_{x\in M}dist(x, p)$ and $dist(M, \partial B_{R}(p))$ such that $$\label{estH-2}
\max_{X\in M,i=1,...,n}|\kappa_i(X)|\leq C.$$

In the proof of Theorem 2, we used $\Phi=\rho^2$ in cases (2) and (3), where $\rho$ is the distance function of the ambient space $
    """,
    r"""
     then runs nouveau drivers?

The biggest strength of ZaReason's Mediabox is, paradoxically, the easiest thing to pick at: It's a box that a hobbyist could build out of stock parts. That really is a strength, though—it's pretty much an ideal build, consisting of the right parts competently assembled, that you didn't have to choose and buy individually and put together yourself... and that won't be impossible to work on due to proprietary parts nobody but the OEM ever made. If you need an HTPC or gaming machine, it's hard to go wrong here.

Price as reviewed: US $1,055

Ubuntu Yakkety Yak (16.10 intermediate release)

Intel i3-7100 CPU at 3.9 GHz

16GB DDR4 SDRAM

GeForce GTX 1060 video

100GB NVMe SSD

DVD-RW

Realtek RTL8168 gigabit Ethernet

Intel 8260 802.11ac WiFi

System76 Oryx Pro

System76's Oryx Pro laptop is not an airy, lightweight consumer device. It's an intimidating, portable battle station with an undeniable gravitas to go along with its all-aluminum-alloy, 6 lb. heft; it wants you to know it is a serious machine being used by a serious developer/admin/gamer/executive, and it gets that message across loud and clear.

There's no way to get across how bright and bold the keyboard backlighting on the Oryx Pro really is without a picture. Unfortunately, while there's probably a way of getting a decent-looking shot of an open laptop with a bright screen and keyboard backlight, I don't know how to!

Oryx Pro's right side features headphone/line/mic audio jacks, a Compact Flash reader, a MultiMedia Card (MMC) reader, two USB3-C (the new symmetrical USB interface) ports, another standard USB3, a Realtek 8168 gigabit Ethernet jack, and a Kensington security lock port.

The benefits of Oryx Pro's no-compromises hefty build don't stop at its menacing good looks: This is without a doubt the most solid feeling laptop I've ever used.

The screen hinges are very firm and tight and feel like they'll stay that way for a long time. There's zero flex in the chassis or under the keyboard as you pick the machine up, move it around, or type on it, no matter how enthusiastic you get. Finally, while 6 lbs is undeniably heavy, the Oryx Pro isn't as bulky as you might think—it's a full size 15.6" laptop, and no doubt about it, but it isn't one bit thicker than it needs to be to fit its full-size Ethernet jack on the side.

The USB3 ports are spaced all the way around the laptop, so it's pretty easy to accommodate wherever you set up and whichever thing has the short cord (or the really long back sticking out). The touchpad and keyboard both feel really nice, and I certainly appreciate the full layout with a number pad on the right. The backlighting will accommodate pretty much anything you want it to do; you can change its color through white/red/blue/green/cyan, and turn its brightness up or down through four or five levels, by using the Fn key with clearly-marked controls on the number-pad.

Finally, the Oryx Pro features an M.2 PCIe system drive with bays for not just one but two additional, optional 2.5" SATA drives if you need them. This is a pretty amazing feature if you need to do demos or move client data around in large quantities, and I'm not sure where except System76 you'd find it.

Unlike the ZaReason Mediabox, the Oryx Pro boots into Canonical's standard "first login" setup for desktop Ubuntu—the OS is already installed, but no user accounts are set until you create your own and give it a password. This does mean you need to spend an extra 15 or 20 seconds before you can start playing... but it also means there's no default system account lurking and waiting for someone to potentially exploit it. Proprietary drivers for its CPU and GPU are pre-installed and operational.

The Oryx Pro starts at US $1,399, and configuration as reviewed was US $1,763. If you want the thinnest, lightest thing you can find or the longest battery life (my Oryx Pro was good for about three hours of moderate use, fully charged), this isn't the machine for you. But if what you really want is a hard-core desktop machine that you
    """,
    """
    Now the real question is, will the model respond with this prompt?
    """
]

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from transformers import TFAutoModelForMaskedLM
from transformers import AutoTokenizer

aug_model_checkpoint = "distilbert-base-uncased"
aug_model = TFAutoModelForMaskedLM.from_pretrained(aug_model_checkpoint)
aug_tokenizer = AutoTokenizer.from_pretrained(aug_model_checkpoint)

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForMaskedLM: ['activation_13']
- This IS expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertForMaskedLM were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForMaskedLM for predictions without further training.


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import copy
from nltk.corpus import stopwords
from string import punctuation
import re

class ContextualAugmenter:
  def __init__(self, model, tokenizer, neuron_model):
    self.model = model
    self.tokenizer = tokenizer
    self.neuron_model = neuron_model
    self.stops = set(stopwords.words('english'))
    self.punctuation_set = set(punctuation)
    self.to_strip = " " + punctuation
    # self.word_tokenizer = re.compile("[ \[\]\(\)]")
    self.word_tokenizer = re.compile(r"\b")
    

  def augment(self, text, n=5):
    tokens = self.word_tokenizer.split(text)
    # tokens = text.split(" ")
    # print(tokens)
    masked_tokens = copy.deepcopy(tokens)
    all_new_texts = []
    all_positions = []

    neuron_tokens = self.neuron_model.to_str_tokens(text)

    for i, token in enumerate(tokens):
      if token in {"", " "} or token.strip(self.to_strip).lower() in self.stops:
        continue
      masked_tokens[i] = "[MASK]"
      before = tokens[:i]
      before_text = "".join(before)
      position = len(before_text)
      # print(masked_tokens)
      masked_text = "".join(masked_tokens)
      inputs = self.tokenizer(masked_text, return_tensors="np")
      token_logits = self.model(**inputs).logits
      # Find the location of [MASK] and extract its logits
      mask_token_index = np.argwhere(inputs["input_ids"] == self.tokenizer.mask_token_id)[0, 1]
      mask_token_logits = token_logits[0, mask_token_index, :]
      # Pick the [MASK] candidates with the highest logits
      # We negate the array before argsort to get the largest, not the smallest, logits
      top_tokens = np.argsort(-mask_token_logits).tolist()

      new_texts = []
      positions = []
      for j, top_token in enumerate(top_tokens):        
        candidate_token = self.tokenizer.decode([top_token])
        normalised_candidate = candidate_token.strip(self.to_strip).lower() if candidate_token not in self.punctuation_set else candidate_token
        normalised_token = token.strip(self.to_strip).lower() if token not in self.punctuation_set else token
        # print("token, top_token", [normalised_token], [normalised_candidate])
        
        if normalised_candidate == normalised_token:
          continue
        new_text = masked_text.replace(self.tokenizer.mask_token, candidate_token)
        new_texts.append(new_text)
        positions.append(position)
        # print(len(new_texts), j, n)   
        if len(new_texts) >= n or j >= n + 5:
          break 

      all_new_texts.extend(new_texts)
      all_positions.extend(positions)
      masked_tokens[i] = token
    
    return all_new_texts, all_positions

In [ ]:
contextual_aug = ContextualAugmenter(aug_model, aug_tokenizer, model)

In [ ]:
contextual_aug.augment("This is a test.", n=2)

(['This is a list.',
  'This is a timeline.',
  'This is a test:',
  'This is a test!'],
 [10, 10, 14, 14])

In [ ]:
augs = [
    contextual_aug
]

In [ ]:
from pprint import pprint
from collections import defaultdict
from string import punctuation
import re
import copy

splitter = re.compile("[\.!\\n]")

def sentence_tokenizer(str_tokens):
  sentences = []
  sentence = []
  sentence_to_token_indices = defaultdict(list)
  token_to_sentence_indices = {}
  
  for i, str_token in enumerate(str_tokens):
    sentence.append(str_token)
    sentence_to_token_indices[len(sentences)].append(i)
    token_to_sentence_indices[i] = len(sentences)
    # if splitter.search(str_token) is not None or i + 1 == len(str_tokens):
    if str_token in {".", "\n"} or i + 1 == len(str_tokens):
      sentences.append(sentence)
      sentence = []    

  return sentences, sentence_to_token_indices, token_to_sentence_indices


def prune(model, layer, neuron, prompt, max_length=1024, proportion_threshold=-0.5, window=4):
  prepend_bos = True
  tokens = model.to_tokens(prompt, prepend_bos=prepend_bos)
  str_tokens = model.to_str_tokens(prompt, prepend_bos=prepend_bos)

  # print(tokens)
  # print(str_tokens)
  # print(tokens.shape)

  if len(tokens[0]) > max_length:
    tokens = tokens[0, :max_length].unsqueeze(0)

  # print(tokens.shape)
  # print(prompt)
  # print("tokens", tokens)
  # print("str_tokens", str_tokens)

  logits, cache = model.run_with_cache(tokens)
  activations = cache[layer][0, :, neuron]

  initial_max = torch.max(activations).cpu().item()
  initial_argmax = torch.argmax(activations).cpu().item()

  # print("initial_max", initial_max)
  # print("initial_argmax", initial_argmax)
  
  sentences, sentence_to_token_indices, token_to_sentence_indices = sentence_tokenizer(str_tokens)

  max_sentence_index = token_to_sentence_indices[initial_argmax]
  sentence = sentences[max_sentence_index]

  offset = sentence_to_token_indices[max_sentence_index][0]
  max_token_index = initial_argmax - offset
  
  shortest_prompt = sentence
  shortest_successful_prompt = None
  for i, index in enumerate([len(sentence) - min((max_token_index + window), len(sentence)), max(0, max_token_index - window)]):
    sentence = shortest_prompt
    for j in range(index):
      if i == 0:
        truncated_prompt = shortest_prompt[:-1]
      else:
        truncated_prompt = shortest_prompt[1:]

      # print("shortest_prompt", shortest_prompt)
      # print(j)
      # print("truncated_prompt", truncated_prompt)

      joined = "".join(truncated_prompt)
      # joined = "".join(sentence)

      # print("\n")
      # print([joined])
      # print(model.to_str_tokens(joined, prepend_bos=prepend_bos))
      
      sentence_tokens = model.to_tokens(joined, prepend_bos=prepend_bos)
      logits, cache = model.run_with_cache(sentence_tokens)
      activations = cache[layer][0, :, neuron]
      sentence_argmax = torch.argmax(activations).cpu().item() + offset
      if prepend_bos:
        sentence_argmax -= 1
      if i == 1:
        sentence_argmax += j + 1
      sentence_max = torch.max(activations).cpu().item()

      # print(sentence_max)
      # print(sentence_argmax)  
      shortest_prompt = truncated_prompt

      if sentence_argmax == initial_argmax and (sentence_max - initial_max) / initial_max > proportion_threshold:        
        shortest_successful_prompt = copy.deepcopy(truncated_prompt)
        
  return "".join(shortest_successful_prompt) if shortest_successful_prompt is not None else None

def augment(model, layer, index, prompt, augs, max_length=1024, inclusion_threshold=-0.5, exclusion_threshold=-0.8, n=1):
  prepend_bos = True
  tokens = model.to_tokens(prompt, prepend_bos=prepend_bos)
  str_tokens = model.to_str_tokens(prompt, prepend_bos=prepend_bos)

  if len(tokens[0]) > max_length:
    tokens = tokens[0, :max_length].unsqueeze(0)

  # print(tokens)
  # print(str_tokens)

  logits, cache = model.run_with_cache(tokens)
  activations = cache[layer][0, :, index]

  initial_max = torch.max(activations).cpu().item()
  initial_argmax = torch.argmax(activations).cpu().item()
  max_char_position = len("".join(str_tokens[int(prepend_bos):initial_argmax]))
  # print("initial_max", initial_max)
  # print("initial_argmax", initial_argmax)

  positive_prompts = [(prompt, initial_max)]
  negative_prompts = []
  # print("starting prompt", prompt)
  for aug in augs:
    aug_prompts, aug_positions = aug.augment(prompt, n=n)
    aug_tokens = model.to_tokens(aug_prompts, prepend_bos=prepend_bos)
    aug_logits, aug_cache = model.run_with_cache(aug_tokens)
    all_aug_activations = aug_cache[layer][:, :, index]

    for aug_prompt, char_position, aug_activations in zip(aug_prompts, aug_positions, all_aug_activations):
      aug_max = torch.max(aug_activations).cpu().item()
      aug_argmax = torch.argmax(aug_activations).cpu().item()
      # print(aug_max)
      # print("actual aug_argmax", aug_argmax)

      # print(str_tokens)
      # print(model.to_str_tokens(aug_prompt, prepend_bos=prepend_bos))
      # print(char_position, max_char_position, aug_argmax)
      if char_position < max_char_position:
        new_str_tokens = model.to_str_tokens(aug_prompt, prepend_bos=prepend_bos)
        aug_argmax += len(new_str_tokens) - len(str_tokens)
        # print("len(new_str_tokens)", len(new_str_tokens))
        # print("len(str_tokens)", len(str_tokens))
        # print("adjusted aug_argmax", aug_argmax)

      proportion_drop = (aug_max - initial_max) / initial_max

      if proportion_drop > inclusion_threshold:
        positive_prompts.append((aug_prompt, aug_max, proportion_drop))
      elif proportion_drop < exclusion_threshold:
        negative_prompts.append((aug_prompt, aug_max, proportion_drop))

  return positive_prompts, negative_prompts

In [ ]:
for snippet in snippets_3_1:
  pruned_prompt = prune(model, "blocks.3.mlp.hook_mid", 1, snippet, window=3)
  print([pruned_prompt])
  if pruned_prompt is None:
    continue
  positive_prompts, negative_prompts = augment(model, "blocks.3.mlp.hook_mid", 1, pruned_prompt, augs, n=4)  
  print("positive", positive_prompts)
  print("negative", negative_prompts)
  print("\n")

[" The thought didn't even cross my mind"]
positive [(" The thought didn't even cross my mind", 0.5641489624977112), (" The words didn't even cross my mind", 0.28522446751594543, -0.494416392696809), (" The question didn't even cross my mind", 0.3628478944301605, -0.3568225441314489), (" The idea didn't even cross my mind", 0.4032658338546753, -0.2851784534544608), (' The thought didn’t even cross my mind', 0.5588880777359009, -0.009325346870298724), (' The thought didn`t even cross my mind', 0.4473706781864166, -0.20699902343925403), (' The thought didn=t even cross my mind', 0.3544965386390686, -0.3716260026969263), (' The thought didn′t even cross my mind', 0.3767401874065399, -0.33219732295782006), (" The thought didn't immediately cross my mind", 0.4713052809238434, -0.16457299001811862), (" The thought didn't completely cross my mind", 0.48807471990585327, -0.13484779313436485), (" The thought didn't quite cross my mind", 0.5182143449783325, -0.08142285207085713), (" The thought 

In [ ]:
for snippet in snippets_3_2:
  pruned_prompt = prune(model, "blocks.3.mlp.hook_mid", 2, snippet, window=3)
  print([pruned_prompt])
  print("\n")
  positive_prompts, negative_prompts = augment(model, "blocks.3.mlp.hook_mid", 2, pruned_prompt, augs, n=4)  
  print("positive", positive_prompts)
  print("negative", negative_prompts)
  print("\n")

[' am not a native English speaker']


positive [(' am not a native English speaker', 0.28990551829338074), (' am not a native american speaker', 0.28990522027015686, -1.0280012109854266e-06), (' am not a native hawaiian speaker', 0.28990522027015686, -1.0280012109854266e-06), (' am not a native indian speaker', 0.28990522027015686, -1.0280012109854266e-06), (' am not a native language speaker', 0.28990522027015686, -1.0280012109854266e-06), (' am not a native English ?', 0.28990522027015686, -1.0280012109854266e-06), (' am not a native English .', 0.28990522027015686, -1.0280012109854266e-06), (' am not a native English language', 0.28990522027015686, -1.0280012109854266e-06), (' am not a native English ;', 0.28990522027015686, -1.0280012109854266e-06)]
negative [(' am not a fluent English speaker', 0.019566699862480164, -0.9325066318928124), (' am not a proper English speaker', 0.023074451833963394, -0.9204069933894382), (' am not a true English speaker', 0.021251285448670387, -0.926

In [ ]:
for snippet in snippets_3_912:
  pruned_prompt = prune(model, "blocks.3.mlp.hook_mid", 912, snippet, window=2, proportion_threshold=-0.6)
  print([pruned_prompt])
  if pruned_prompt is None:
    continue  
  print("\n")
  positive_prompts, negative_prompts = augment(model, "blocks.3.mlp.hook_mid", 912, pruned_prompt, augs, n=2, inclusion_threshold=-0.4, exclusion_threshold=-0.7)  
  print("positive", positive_prompts)
  print("negative", negative_prompts)
  print("\n")

[' was put on the IUCN Red List under']


positive [(' was put on the IUCN Red List under', 0.7422030568122864), (' was placed on the IUCN Red List under', 0.6806896328926086, -0.08287950764292709), (' was listed on the IUCN Red List under', 0.709409773349762, -0.04418370843603019)]
negative [(' was put on the iaaf Red List under', 0.21633580327033997, -0.7085220799285197), (' was put on the nhl Red List under', 0.20250383019447327, -0.7271584530192937), (' was put on the IUCN threatened List under', 0.1870197206735611, -0.7480208159249592), (' was put on the IUCN Red ##lists under', 0.04687381535768509, -0.936845025188922)]


[' It was added to the National Register of']


positive [(' It was added to the National Register of', 0.502929151058197), (' It was add to the National Register of', 0.4071315824985504, -0.1904792521135075), (' It was named to the National Register of', 0.33700308203697205, -0.3299193706948688), (' It was added to the National list of', 0.44478467106819153, -0.

In [ ]:
for snippet in snippets_4_912:
  pruned_prompt = prune(model, "blocks.4.mlp.hook_mid", 912, snippet, window=5, proportion_threshold=-0.6)
  print([pruned_prompt])
  if pruned_prompt is None:
    continue  
  print("\n")
  positive_prompts, negative_prompts = augment(model, "blocks.4.mlp.hook_mid", 912, pruned_prompt, augs, n=2, inclusion_threshold=-0.4, exclusion_threshold=-0.7)  
  print("positive", positive_prompts)
  print("negative", negative_prompts)
  print("\n")

['.](bjc201256f1){#fig1']


positive [('.](bjc201256f1){#fig1', 2.9047985076904297), ('.](bjc201256f1)fig1', 2.904808521270752, 3.447254704846052e-06), ('.](bjc201256f1]fig1', 2.904808521270752, 3.447254704846052e-06), ('.](bjc201256f1){#;', 2.904808521270752, 3.447254704846052e-06), ('.](bjc201256f1){#}', 2.904808521270752, 3.447254704846052e-06)]
negative [('(bjc201256f1){#fig1', 0.039379484951496124, -0.9864432989595529), ('#bjc201256f1){#fig1', 0.04763243347406387, -0.9836021557612491), ('.](\\){#fig1', 0.025366412475705147, -0.9912674106625475), ('.](thumb){#fig1', 0.024784769862890244, -0.9914676457601886)]


['.](bjc201156f1){#fig1']


positive [('.](bjc201156f1){#fig1', 2.9739015102386475), ('.](bjc201156f1)fig1', 2.973909378051758, 2.64561993168368e-06), ('.](bjc201156f1]fig1', 2.973909378051758, 2.64561993168368e-06), ('.](bjc201156f1){#;', 2.973909378051758, 2.64561993168368e-06), ('.](bjc201156f1){#}', 2.973909378051758, 2.64561993168368e-06)]
negative [('(bjc201156f1){#fig

In [ ]:
for snippet in snippets_4_0:
  pruned_prompt = prune(model, "blocks.4.mlp.hook_mid", 0, snippet, window=5, proportion_threshold=-0.6)
  print([pruned_prompt])
  if pruned_prompt is None:
    continue  
  print("\n")
  positive_prompts, negative_prompts = augment(model, "blocks.4.mlp.hook_mid", 0, pruned_prompt, augs, n=2, inclusion_threshold=-0.4, exclusion_threshold=-0.7)  
  print("positive", positive_prompts)
  print("negative", negative_prompts)
  print("\n")

[' manifolds are smooth, connected, Hausdorff, with non-degenerate metric']


positive [(' manifolds are smooth, connected, Hausdorff, with non-degenerate metric', 0.7242268323898315), (' they are smooth, connected, Hausdorff, with non-degenerate metric', 0.5182992219924927, -0.2843413157143198), (' surfaces are smooth, connected, Hausdorff, with non-degenerate metric', 0.47448647022247314, -0.34483721259436845), (' manifolds are compact, connected, Hausdorff, with non-degenerate metric', 0.5818287134170532, -0.19662088258023738), (' manifolds are continuous, connected, Hausdorff, with non-degenerate metric', 0.5475104451179504, -0.24400695937865982), (' manifolds are smooth, compact, Hausdorff, with non-degenerate metric', 0.4534768760204315, -0.3738469002535696), (' manifolds are smooth, convex, Hausdorff, with non-degenerate metric', 0.48599180579185486, -0.32895084239262384), (' manifolds are smooth, connectedtoHausdorff, with non-degenerate metric', 0.4778589904308319, -0.34018049